## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [21]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [22]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [23]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'/SNWD_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [24]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3
0,-0.147380,-0.054716,0.187917,0.154729
1,-0.205817,0.020798,-0.205021,0.109136
2,-0.134367,-0.177230,0.240033,0.117359
3,-0.179602,0.017340,-0.203439,0.081376
4,-0.084922,-0.156979,0.185165,-0.024159


In [25]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'US1COJF0024', u'USC00052790', u'USC00051660', u'US1COEG0008', u'USC00050214']


,0,1,2,3
station,,,,
US1COJF0024,-0.147380,-0.054716,0.187917,0.154729
USC00052790,-0.205817,0.020798,-0.205021,0.109136
USC00051660,-0.134367,-0.177230,0.240033,0.117359
US1COEG0008,-0.179602,0.017340,-0.203439,0.081376
USC00050214,-0.084922,-0.156979,0.185165,-0.024159


In [48]:
a=set(Eig.reset_index()['station']).intersection(['USC00052790',
 'USC00051660',
 'US1COEG0008',
 'USC00050214',
 'USC00053508',
 'USC00053530',
 'USC00051829',
 'USC00050674',
 'US1COJF0276',
 'USC00055507',
 'US1COJF0104',
 'US1COJF0106',
 'USC00051743',
 'USC00053261'])

In [61]:
Eig[Eig.index=='US1COJF0104']

,0,1,2,3
station,,,,
US1COJF0104,-0.246347,0.07176,-0.140942,0.022424


In [51]:
a

{'US1COEG0008',
 'US1COJF0104',
 'US1COJF0106',
 'US1COJF0276',
 'USC00050214',
 'USC00050674',
 'USC00051660',
 'USC00051743',
 'USC00051829',
 'USC00052790',
 'USC00053261',
 'USC00053508',
 'USC00053530',
 'USC00055507'}

In [56]:
df = pd.DataFrame(Eig[Eig.index=='US1COJF0024'].reset_index())
for i in a:
    pd.concat([df,Eig[Eig.index==i].reset_index()],axis=1)

In [57]:
df

,station,0,1,2,3
0,US1COJF0024,-0.14738,-0.054716,0.187917,0.154729


In [14]:
table=Eig.join(Stations,how='left')

In [15]:
table=table[['name','latitude','longitude','elevation',0,1,2,3]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
US1COJF0024,CONIFER 4.2 ENE,39.5346,-105.2273,2364.9,-0.147380,-0.054716,0.187917,0.154729
USC00052790,EVERGREEN,39.6381,-105.3150,2129.0,-0.205817,0.020798,-0.205021,0.109136
USC00051660,CLIMAX,39.3672,-106.1897,3442.4,-0.134367,-0.177230,0.240033,0.117359
US1COEG0008,WOLCOTT 2.3 N,39.7332,-106.6806,2201.9,-0.179602,0.017340,-0.203439,0.081376


In [16]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [17]:
min_lat,max_lat,min_long,max_long = box = (39.1014, 39.8143, -108.7339, -105.1208)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

In [10]:
fairplay
aspen
vail
denver
glenwood springs
grand Junction

SyntaxError: invalid syntax (<ipython-input-10-e7db4954baa7>, line 5)

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.